In [1]:
# Run a mock of fractal-demos/examples/01, with an additional illumination correction task.

"""
NOTES
1. The reference cycle is currently identified by its name (the last part of the
   OME-Zarr image path). We can then change this into using some zarr metadata.
2. We don't know yet how the to include two versions of the same reference-cycle
   image in the image list (where one has registration=True). This means that the
   the current version does not support (yet) running another task after
   registration.
   In order to have a consistent prototype, here we follow the naive approach of
   also including a dummy (ref_path, ref_path) pair in the parallelization list
   produced by the init-registration task.
3. This mock assumes that the "registration" task takes care of all the relevant
   steps, including: computing shifts, preparing new ROI tables, writing new
   image data.
   The mock task simulates the writing of image data, but in principle it should
   also include writing of new ROI tables. We haven't explored yet possible
   image-list-related glitches occurring when considering tables.
"""

'\nNOTES\n1. The reference cycle is currently identified by its name (the last part of the\n   OME-Zarr image path). We can then change this into using some zarr metadata.\n2. We don\'t know yet how the to include two versions of the same reference-cycle\n   image in the image list (where one has registration=True). This means that the\n   the current version does not support (yet) running another task after\n   registration.\n   In order to have a consistent prototype, here we follow the naive approach of\n   also including a dummy (ref_path, ref_path) pair in the parallelization list\n   produced by the init-registration task.\n3. This mock assumes that the "registration" task takes care of all the relevant\n   steps, including: computing shifts, preparing new ROI tables, writing new\n   image data.\n   The mock task simulates the writing of image data, but in principle it should\n   also include writing of new ROI tables. We haven\'t explored yet possible\n   image-list-related glit

In [2]:
# Preliminary imports and auxiliary-function definition

from pathlib import Path
from devtools import debug
import tempfile

from fractal_server.app.runner.v2.models import Dataset, WorkflowTask
from fractal_server.app.runner.v2.runner import execute_tasks_v2
from fractal_server.images import SingleImage

from concurrent.futures import ThreadPoolExecutor

import sys
sys.path.append("../tests/v2/04_runner")
from fractal_tasks_core_mock import TASK_LIST

executor = ThreadPoolExecutor()


def image_data_exist_on_disk(image_list: list[SingleImage]):
    """
    Given an image list, check whether mock data were written to disk.
    """
    prefix = "[image_data_exist_on_disk]"
    all_images_have_data = True
    for image in image_list:
        if (Path(image.path) / "data").exists():
            print(f"{prefix} {image.path} contains data")
        else:
            print(f"{prefix} {image.path} does *not* contain data")
            all_images_have_data = False
    return all_images_have_data

In [3]:
# Create temporary directory for mocked zarrs
tmp_path = Path(tempfile.mkdtemp())
zarr_dir = (tmp_path / "zarr_dir").as_posix().rstrip("/")
print(f"{zarr_dir=}")

zarr_dir='/tmp/tmp50ro_aoj/zarr_dir'


In [4]:
# Run create-ome-zarr-multiplex
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["create_ome_zarr_multiplex"],
            args=dict(image_dir="/tmp/input_images", zarr_dir=zarr_dir),
        )
    ],
    dataset=Dataset(),
    executor=executor,
)

# Print current dataset information
debug(dataset)

# We have 6 images (two wells, three cycles)
assert len(dataset.images) == 6
assert dataset.image_paths == [
    f"{zarr_dir}/my_plate.zarr/A/01/0",
    f"{zarr_dir}/my_plate.zarr/A/01/1",
    f"{zarr_dir}/my_plate.zarr/A/01/2",
    f"{zarr_dir}/my_plate.zarr/A/02/0",
    f"{zarr_dir}/my_plate.zarr/A/02/1",
    f"{zarr_dir}/my_plate.zarr/A/02/2",
]

# Image data do not exit on disk yet
assert not image_data_exist_on_disk(dataset.images)

[create_ome_zarr_multiplex] START
[create_ome_zarr_multiplex] image_dir='/tmp/input_images'
[create_ome_zarr_multiplex] zarr_dir='/tmp/tmp50ro_aoj/zarr_dir'
[create_ome_zarr_multiplex] zarr_path='/tmp/tmp50ro_aoj/zarr_dir/my_plate.zarr'
[create_ome_zarr_multiplex] END
/tmp/ipykernel_99491/1187602282.py:14 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_multiplex',
        ],
        images=[
            SingleImage(
                path='/tmp/tmp50ro_aoj/zarr_dir/my_plate.zarr/A/01/0',
                attributes={
                    'well': 'A01',
                    'acquisition': 0,
                    'plate': 'my_plate.zarr',
                    'data_dimensionality': 3,
                },
            ),
            SingleImage(
                path='/tmp/tmp50ro_aoj/zarr_dir/my_plate.zarr/A/01/1',
                attributes={
                    'well': 'A01',
                    'acquisition': 1,
                    'plate': 'my_pla

In [5]:
# Run yokogawa-to-zarr
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(task=TASK_LIST["yokogawa_to_zarr"]),
    ],
    dataset=dataset,
    executor=executor,
)

# Print current dataset information
debug(dataset)

# We still have the same 6 images (two wells, three cycles)
assert len(dataset.images) == 6
assert dataset.image_paths == [
    f"{zarr_dir}/my_plate.zarr/A/01/0",
    f"{zarr_dir}/my_plate.zarr/A/01/1",
    f"{zarr_dir}/my_plate.zarr/A/01/2",
    f"{zarr_dir}/my_plate.zarr/A/02/0",
    f"{zarr_dir}/my_plate.zarr/A/02/1",
    f"{zarr_dir}/my_plate.zarr/A/02/2",
]

# Image data now exist on disk
assert image_data_exist_on_disk(dataset.images)

[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmp50ro_aoj/zarr_dir/my_plate.zarr/A/01/0'
[yokogawa_to_zarr] source_data='/tmp/input_images/A_01_0.tif'
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmp50ro_aoj/zarr_dir/my_plate.zarr/A/01/2'
[yokogawa_to_zarr] source_data='/tmp/input_images/A_01_2.tif'
[yokogawa_to_zarr] END
[yokogawa_to_zarr] END
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmp50ro_aoj/zarr_dir/my_plate.zarr/A/02/1'
[yokogawa_to_zarr] source_data='/tmp/input_images/A_02_1.tif'
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmp50ro_aoj/zarr_dir/my_plate.zarr/A/02/2'
[yokogawa_to_zarr] source_data='/tmp/input_images/A_02_2.tif'
[yokogawa_to_zarr] END
[yokogawa_to_zarr] END
/tmp/ipykernel_99491/334051424.py:11 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_multiplex',
            'yokogawa_to_zarr',
        ],
        images=[
            SingleImage(
                path='/tmp/tmp50ro_aoj/

In [6]:
# Run init-registration
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["init_registration"],
            args={"ref_cycle_name": "0"},
        )
    ],
    dataset=dataset,
    executor=executor,
)

# Print current dataset information
debug(dataset)

# The dataset now includes a custom parallelization list
assert dataset.parallelization_list is not None

# We still have the same 6 images, because the init-registration task only
# creates a parallelization list (rather than adding new images to the list)
assert len(dataset.images) == 6

/tmp/ipykernel_99491/2034724189.py:14 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_multiplex',
            'yokogawa_to_zarr',
            'init_registration',
        ],
        images=[
            SingleImage(
                path='/tmp/tmp50ro_aoj/zarr_dir/my_plate.zarr/A/01/0',
                attributes={
                    'well': 'A01',
                    'acquisition': 0,
                    'plate': 'my_plate.zarr',
                    'data_dimensionality': 3,
                },
            ),
            SingleImage(
                path='/tmp/tmp50ro_aoj/zarr_dir/my_plate.zarr/A/01/1',
                attributes={
                    'well': 'A01',
                    'acquisition': 1,
                    'plate': 'my_plate.zarr',
                    'data_dimensionality': 3,
                },
            ),
            SingleImage(
                path='/tmp/tmp50ro_aoj/zarr_dir/my_plate.zarr/A/01/2',
                at

In [7]:
# Run registration with overwrite_input=True (the default)
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["registration"],
            args={"overwrite_input": False}
        ),
    ],
    dataset=dataset,
    executor=executor,
)

# Print current dataset information
debug(dataset)

# We now have 12 images (6 raw, and 6 corrected) 
assert len(dataset.images) == 12
assert dataset.image_paths == [
    f"{zarr_dir}/my_plate.zarr/A/01/0",
    f"{zarr_dir}/my_plate.zarr/A/01/1",
    f"{zarr_dir}/my_plate.zarr/A/01/2",
    f"{zarr_dir}/my_plate.zarr/A/02/0",
    f"{zarr_dir}/my_plate.zarr/A/02/1",
    f"{zarr_dir}/my_plate.zarr/A/02/2",
    f"{zarr_dir}/my_plate.zarr/A/01/0_r",
    f"{zarr_dir}/my_plate.zarr/A/01/1_r",
    f"{zarr_dir}/my_plate.zarr/A/01/2_r",
    f"{zarr_dir}/my_plate.zarr/A/02/0_r",
    f"{zarr_dir}/my_plate.zarr/A/02/1_r",
    f"{zarr_dir}/my_plate.zarr/A/02/2_r",
]

# The first-image metadata has not changed
assert dataset.images[6].dict() == {
    "path": f"{zarr_dir}/my_plate.zarr/A/01/0_r",
    "attributes": {
        "well": "A01",
        "acquisition": 0,
        "plate": "my_plate.zarr",
        "data_dimensionality": 3,
        "registration": True,
    },
}

# The custom parallelization list is not present any more
assert dataset.parallelization_list is None

# Images 0-5 are the raw ones, with no "registration" attribute
for image in dataset.images[:6]:
    assert image.attributes.get("registration") is None

# Images 6-11 are the registered ones
for image in dataset.images[6:]:
    assert image.attributes["registration"] is True

[registration] START
[registration] path='/tmp/tmp50ro_aoj/zarr_dir/my_plate.zarr/A/02/0'
[registration] overwrite_input=False
[registration] new_path='/tmp/tmp50ro_aoj/zarr_dir/my_plate.zarr/A/02/0_r'
[registration] out={'added_images': [{'path': '/tmp/tmp50ro_aoj/zarr_dir/my_plate.zarr/A/02/0_r'}]}
[registration] END
[registration] START
[registration] path='/tmp/tmp50ro_aoj/zarr_dir/my_plate.zarr/A/02/2'
[registration] overwrite_input=False
[registration] new_path='/tmp/tmp50ro_aoj/zarr_dir/my_plate.zarr/A/02/2_r'
[registration] out={'added_images': [{'path': '/tmp/tmp50ro_aoj/zarr_dir/my_plate.zarr/A/02/2_r'}]}
[registration] END
/tmp/ipykernel_99491/3472021409.py:14 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_multiplex',
            'yokogawa_to_zarr',
            'init_registration',
            'registration',
        ],
        images=[
            SingleImage(
                path='/tmp/tmp50ro_aoj/zarr_dir/my_plate.zarr/A/01/

In [8]:
executor.shutdown()